In [1]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm
import torch
from datasets import load_dataset
!pip install evaluate
import evaluate
import logging
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer,AutoModel,LEDConfig,LEDForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from nltk import sent_tokenize
!pip install rouge-score
!pip install bert_score
#from peft import get_peft_model, LoraConfig

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
#ms2 = load_dataset("allenai/mslr2022","ms2")
cochrane=load_dataset("allenai/mslr2022","cochrane")

In [4]:
"""ms2_train=ms2["train"].to_pandas()
ms2_test=ms2["test"].to_pandas()
ms2_val=ms2["validation"].to_pandas()"""
train_df=cochrane["train"].to_pandas()
test_df=cochrane["test"].to_pandas()
val_df=cochrane["validation"].to_pandas()

In [5]:
all_df=[train_df,test_df,val_df]
"""for df in [ms2_train,ms2_test,ms2_val]:
    df.drop(columns=['background','review_id','pmid'],inplace=True)"""
for df in all_df:
       df.drop(columns=['review_id','pmid'],inplace=True)

def conc_text(tit,abst):
    text=[]
    assert len(tit)==len(abst)
    for i in range(len(abst)):
        if tit[i].endswith(".") or tit[i].endswith("!") or tit[i].endswith("?"):
            text.append(tit[i]+" "+abst[i])
        else : text.append(tit[i]+". "+abst[i])
    return "<doc-sep>".join(text)
for df in all_df:
    df["text"]=df.apply(lambda row: conc_text(row['title'],row['abstract']),axis=1)
    df["text"]=df["text"].astype(str)
    df["target"]=df["target"].astype(str)
    df.drop(columns=['title','abstract'],inplace=True)

In [6]:
train_df.head()

,target,text
0,We found only low quality evidence comparing u...,Aggressive surgical effort and improved surviv...
1,Prophylactic indomethacin has short-term benef...,Prophylactic indomethacin therapy in the first...
2,First aid for phosphorus burns involves the co...,White phosphorus burns and massive hemolysis. ...
3,"Compared with control, there was no evidence o...",Trial of long-term anticoagulant therapy in th...
4,The diversity of psychological interventions i...,Computer-aided retraining of memory and attent...


In [ ]:
tokenizer=AutoTokenizer.from_pretrained("/kaggle/input/tokenizer/primera-tokenizer")
#tokenizer=AutoTokenizer.from_pretrained("allenai/PRIMERA")

In [8]:
class SummarizationDataset(Dataset):
    def __init__(self,tokenizer,max_input_len,max_output_len,data):
        self.tokenizer=tokenizer
        self.src_list=data["text"]
        self.tgt_list=data["target"]
        self.max_input_len=max_input_len
        self.max_output_len=max_output_len
    def __getitem__(self,index):
        src_text=str(self.src_list.loc[index])
        tgt_text=str(self.tgt_list.loc[index])
        src=self.tokenizer(src_text,padding="max_length",truncation=True,max_length=self.max_input_len)
        tgt=self.tokenizer(tgt_text,padding="max_length",truncation=True,max_length=self.max_output_len)
        input_ids=src.input_ids
        global_attention_mask=[0 for _ in range(len(input_ids))]
        global_attention_mask[input_ids==self.tokenizer.vocab["<doc-sep>"]]=1
        global_attention_mask[0]=1
        labels=tgt.input_ids
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels] 
        return {
            "input_ids":torch.tensor(input_ids,dtype=torch.long),
            "attention_mask":torch.tensor(src.attention_mask,dtype=torch.long),
            "global_attention_mask":torch.tensor(global_attention_mask,dtype=torch.long),
            "labels":torch.tensor(labels,dtype=torch.long),
            "labels_str":tgt_text
        }
    def __len__(self):
        return self.src_list.shape[0]
MAX_INPUT_LEN=4096
MAX_OUTPUT_LEN=512
train_dataset=SummarizationDataset(tokenizer,max_input_len=MAX_INPUT_LEN,max_output_len=MAX_OUTPUT_LEN,data=train_df)
val_dataset=SummarizationDataset(tokenizer,max_input_len=MAX_INPUT_LEN,max_output_len=MAX_OUTPUT_LEN,data=val_df)
test_dataset=SummarizationDataset(tokenizer,max_input_len=MAX_INPUT_LEN,max_output_len=MAX_OUTPUT_LEN,data=test_df)

In [16]:
#BASE_MODEL_PATH="allenai/PRIMERA"
MODEL_PATH= "/kaggle/input/finetuned-primera-ckp/checkpoint-2200" #"/kaggle/input/pretrained-primera/checkpoint-7950"
#MODEL_PATH= "./finetuned-primera/checkpoint-2200"
config = LEDConfig.from_pretrained(MODEL_PATH)
config.max_decoder_position_embeddings = 1024  
#config.vocab_size=len(tokenizer)
model = LEDForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    config=config,
    ignore_mismatched_sizes=True
)
model.gradient_checkpointing_enable()

In [17]:
model.config

LEDConfig {
  "_attn_implementation_autoset": true,
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dilation": [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1
  ],
  "attention_dropout": 0.0,
  "attention_mode": "sliding_chunks",
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "autoregressive": false,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_

In [ ]:
"""lora_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj"]
)
model = get_peft_model(model, lora_config)

# Print which parameters are trainable

model.print_trainable_parameters()"""

In [11]:
rouge=evaluate.load("rouge")
bertscore=evaluate.load("bertscore")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    
    # Convert -100 values to tokenizer's pad_token_id safely
    labels_ids = np.where(labels_ids == -100, tokenizer.pad_token_id, labels_ids)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute ROUGE scores
    return compute_score(pred_str,label_str)

def compute_score(pred_str,label_str):
    rouge_output = rouge.compute(
        predictions=pred_str, 
        references=label_str, 
        rouge_types=["rouge2"]#, "rouge1", "rougeL"]
    )

    # Compute BERTScore
    bertscore_output = bertscore.compute(predictions=pred_str, references=label_str, lang="en")

    return {
        "rouge2": round(rouge_output["rouge2"], 4),
        "bertscore":round(np.mean(bertscore_output["f1"]),4)
    }

In [ ]:
model.generation_config

In [ ]:
import copy
gen_config = copy.deepcopy(model.generation_config)
gen_config.update(
    bad_words_ids=[[tokenizer.mask_token_id]],
    num_beams=4,
    max_new_tokens=156,
    force_words_ids=[[tokenizer.eos_token_id]],
    length_penalty=1.2,             
    repetition_penalty=1.2,         
    no_repeat_ngram_size=3,
)
gen_config.validate()

In [ ]:

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
batch_size=4
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./finetuned-primera",
    logging_dir="./logs/fine-tuned/",
    save_strategy="steps",
    save_steps=20,
    logging_strategy="steps",
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-5,
    #warmup_steps=3000,
    #lr_scheduler_type="linear",
    report_to="tensorboard",
    dataloader_num_workers=4,
    disable_tqdm=False,
    ddp_find_unused_parameters=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [12]:
torch.cuda.empty_cache()

In [ ]:
trainer.train(resume_from_checkpoint=False)

In [ ]:
trainer.save_model("./finetuned-primera")

In [ ]:
def inference(batch_data,model,tokenizer,temperature):
    input_ids=batch_data["input_ids"].cuda()
    attention_mask=batch_data["attention_mask"].cuda()
    global_attention_mask=batch_data["global_attention_mask"].cuda()
    target_ids=batch_data["labels"].cuda()
    pred_ids = model.generate(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              global_attention_mask=global_attention_mask, 
                              bad_words_ids=[[tokenizer.mask_token_id]],
                              num_beams=4,
                              max_new_tokens=156,
                              force_words_ids=[[tokenizer.eos_token_id]],
                              length_penalty=1.2,             
                              repetition_penalty=1.2,         
                              no_repeat_ngram_size=3,  
                              temperature=temperature )
    target_ids[target_ids==-100]=tokenizer.pad_token_id
    pred=tokenizer.batch_decode(pred_ids,skip_special_tokens=True)
    target=tokenizer.batch_decode(target_ids,skip_special_tokens=True)
    real_target=batch_data["labels_str"]
    #score_dict=compute_score(pred,target)
    for i in range(len(target)):
        print("i=",i)
        print("Real target: ",real_target[i])
        #print("Target_id: ",target_ids)
        print("Target: ",target[i])
        print("####################################")
        print("Pred_id: ",pred_ids[i])
        print("Pred: ",pred[i])
        print("------------------------------------")
    return pred#,score_dict

In [16]:
TEST_BATCH_SIZE=2
val_dataloader=DataLoader(val_dataset,batch_size=TEST_BATCH_SIZE,shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)
model=model.cuda()

In [ ]:
tokenizer.convert_ids_to_tokens([879])

In [ ]:
idx=3
val_iter=iter(val_dataloader)
for _ in range(idx):
    batch=next(val_iter)
inference(batch,model,tokenizer,1.0)

In [24]:
def inference(batch_data, model, tokenizer, temperature=1.0, device="cuda" if torch.cuda.is_available() else "cpu"):
    try:
        # Move tensors to device
        input_ids = batch_data["input_ids"].to(device)
        attention_mask = batch_data["attention_mask"].to(device)
        global_attention_mask = batch_data["global_attention_mask"].to(device)
        target_ids = batch_data["labels"].to(device)

        # Ensure model is on device and in eval mode
        model = model.to(device)
        model.eval()

        # Generate predictions
        with torch.no_grad():
            pred_ids = model.generate(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              global_attention_mask=global_attention_mask, 
                              bad_words_ids=[[tokenizer.mask_token_id]],
                              num_beams=4,
                              max_new_tokens=156,
                              force_words_ids=[[tokenizer.eos_token_id]],
                              length_penalty=1.2,             
                              repetition_penalty=1.2,         
                              no_repeat_ngram_size=3,
                              temperature=temperature)  
        target_ids[target_ids == -100] = tokenizer.pad_token_id

        # Decode predictions and targets
        pred = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        target = tokenizer.batch_decode(target_ids, skip_special_tokens=True)
        real_target = batch_data["labels_str"]

        # Compute scores
        score_dict = compute_score(pred, real_target)

        return pred, score_dict

    except Exception as e:
        print(f"Error during inference: {str(e)}")
        raise

def compute_score(pred_str, label_str):
    """
    Compute ROUGE-1, ROUGE-2, ROUGE-L, and BERTScore for predictions.
    
    Args:
        pred_str (list): List of predicted texts.
        label_str (list): List of reference texts.
    
    Returns:
        list: List of dictionaries containing metrics for each sample.
    """
    scores = []
    for pred, ref in zip(pred_str, label_str):
        score = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0, "bert_score": 0.0}
        
        # Skip empty predictions or references
        if not pred.strip() or not ref.strip():
            logging.warning(f"Empty prediction or reference detected: pred='{pred}', ref='{ref}'. Setting scores to 0.")
            scores.append(score)
            continue

        # Compute ROUGE scores for this sample
        try:
            rouge_output = rouge.compute(
                predictions=[pred],
                references=[ref],
                rouge_types=["rouge1", "rouge2", "rougeL"],
                use_aggregator=True
            )
            score["rouge1"] = round(rouge_output["rouge1"], 4)
            score["rouge2"] = round(rouge_output["rouge2"], 4)
            score["rougeL"] = round(rouge_output["rougeL"], 4)
        except Exception as e:
            logging.error(f"ROUGE computation failed for pred='{pred}', ref='{ref}': {str(e)}")
            scores.append(score)
            continue

        # Compute BERTScore for this sample
        try:
            bertscore_output = bertscore.compute(
                predictions=[pred],
                references=[ref],
                lang="en"
            )
            score["bert_score"] = round(bertscore_output["f1"][0], 4)
        except Exception as e:
            logging.error(f"BERTScore computation failed for pred='{pred}', ref='{ref}': {str(e)}")
            score["bert_score"] = 0.0

        scores.append(score)

    return scores

In [28]:
# Initialize results
results = {
    "target": [],
    "prediction": [],
    "rouge-1": [],
    "rouge-2": [],
    "rouge-L": [],
    "bert_score": []
}

# Run inference
model.eval()
for batch in tqdm(val_dataloader, desc="Processing test dataset"):
    try:
        pred, score_dict = inference(batch, model, tokenizer, temperature=1.0)
        batch_targets = batch["labels_str"]

        for target, p, scores in zip(batch_targets, pred, score_dict):
            results["target"].append(target)
            results["prediction"].append(p)
            results["rouge-1"].append(scores["rouge1"])
            results["rouge-2"].append(scores["rouge2"])
            results["rouge-L"].append(scores["rougeL"])
            results["bert_score"].append(scores["bert_score"])

    except Exception as e:
        print(f"Error processing batch: {str(e)}")
        continue

# Create DataFrame
df_results = pd.DataFrame(results)

# Compute average scores
avg_scores = {
    "rouge-1": round(df_results["rouge-1"].mean(), 4),
    "rouge-2": round(df_results["rouge-2"].mean(), 4),
    "rouge-L": round(df_results["rouge-L"].mean(), 4),
    "bert_score": round(df_results["bert_score"].mean(), 4)
}
print("Average Scores:", avg_scores)

# Append average scores as a new row
avg_row = pd.DataFrame({
    "target": ["Avg score"],
    "prediction": [""],
    "rouge-1": [avg_scores["rouge-1"]],
    "rouge-2": [avg_scores["rouge-2"]],
    "rouge-L": [avg_scores["rouge-L"]],
    "bert_score": [avg_scores["bert_score"]]
})
df_results = pd.concat([avg_row, df_results], ignore_index=True)

# Save to CSV
output_path = "./val.csv"
df_results.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")

Processing test dataset: 100%|██████████| 235/235 [12:34<00:00,  3.21s/it]

Average Scores: {'rouge-1': 0.2855, 'rouge-2': 0.0641, 'rouge-L': 0.1784, 'bert_score': 0.864}
Results saved to ./val.csv


In [5]:
val_res=pd.read_csv("./test.csv")

In [18]:
val_df=cochrane["test"].to_pandas()

In [19]:
val_df

,review_id,pmid,title,abstract,target
0,CD000220,"[11519502, 6845046]",[Failure of metronidazole to prevent preterm d...,[Infection with Trichomonas vaginalis during p...,
1,CD008120,"[16259547, 18246327, 18007568, 16139813, 12503...",[Adjunctive risperidone in generalized anxiety...,[Although significant advances have been made ...,
2,CD002968,"[9699091, 12351469, 12829654, 11891019, 145782...",[Effects of a short-term circuit weight traini...,[This study assessed the effects of short-term...,
3,CD008472,"[20484064, 19995115, 20484066, 17846333]",[Remote ischemic preconditioning for cerebral ...,[Remote ischemic preconditioning (RIPC) is a p...,
4,CD006373,"[16293396, 15567053, 15963687, 11566828, 10223...",[The effectiveness of a voice treatment approa...,[Teachers are considered the professional grou...,
...,...,...,...,...,...
465,CD001815,"[14581796, 10194995, 12960650, 16737869, 11198...",[Establishing enteral feeding in preterm infan...,"[A prospective, double-blind, randomized, cont...",
466,CD004447,[8747120],[Stimulation of growth hormone secretion in ch...,[X-linked hypophosphatemia is characterized by...,
467,CD003700,"[16280443, 9665113, 9329303]",[Effects of moderate doses of vitamin A as an ...,[Randomized controlled trials have shown incon...,
468,CD003412,"[10570388, 14732655, 9218740, 12224977, 124528...",[Therapeutic response of basal cell carcinoma ...,[Basal cell carcinoma (BCC) responds to interf...,


In [20]:
val_res

,review_id,target,prediction,rouge-1,rouge-2,rouge-L,bert_score
0,avg_score,Avg score,NaN,0.0,0.0,0.0,0.0
1,CD007016,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
2,CD007119,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
3,CD006525,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
4,CD005571,NaN,There is insufficient evidence to determine wh...,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
466,CD004079,NaN,We found no evidence to support the routine us...,0.0,0.0,0.0,0.0
467,CD006576,NaN,There is insufficient evidence to support or r...,0.0,0.0,0.0,0.0
468,CD002918,NaN,There is insufficient evidence to support or r...,0.0,0.0,0.0,0.0
469,CD004182,NaN,The available evidence suggests that topical i...,0.0,0.0,0.0,0.0


In [21]:
# Create a list of review_ids with "avg_score" first, followed by val_df's review_id values
review_ids = ["avg_score"] + val_df['review_id'].tolist()

# Assign it to val_res
val_res['review_id'] = review_ids

# Optional: move 'review_id' to the first column
val_res = val_res[['review_id'] + [col for col in val_res.columns if col != 'review_id']]

In [11]:
val_res

,review_id,target,prediction,rouge-1,rouge-2,rouge-L,bert_score
0,avg_score,Avg score,NaN,0.0,0.0,0.0,0.0
1,CD007016,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
2,CD007119,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
3,CD006525,NaN,There is insufficient evidence to support the ...,0.0,0.0,0.0,0.0
4,CD005571,NaN,There is insufficient evidence to determine wh...,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
466,CD004079,NaN,We found no evidence to support the routine us...,0.0,0.0,0.0,0.0
467,CD006576,NaN,There is insufficient evidence to support or r...,0.0,0.0,0.0,0.0
468,CD002918,NaN,There is insufficient evidence to support or r...,0.0,0.0,0.0,0.0
469,CD004182,NaN,The available evidence suggests that topical i...,0.0,0.0,0.0,0.0


In [22]:
output_path = "./test_res.csv"
val_res.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")

Results saved to ./test_res.csv
